# EE4375-2022: Second Lab Session: Functions, Type Stability and Benchmarking

## Import Packages  

In [1]:
using LinearAlgebra 
using SparseArrays 

using IterativeSolvers
using Preconditioners

using BenchmarkTools
using Profile
using ProfileView

using Plots 

## Section 1: Build Linear System as Sparse From the Start
Motivation 
1. build matrix as sparse directly, i.e., avoid convection from dense to sparse matrix; 
2. use code as building block for more complex code; 
3. profile code in terms of type-stability, memory and CPU usage; 
4. unit test the code (compare with analytical solution or rate of converge); 
5. upload the code of github to work with github actions; 

### Section 1.1: Build Coefficient Matrix 
We specify the type of the input argument N to providing the compiler more informartion to perform required optimizations. In the following, we 
1. test the code on small input values; 
2. verify the type stability of the code;
3. benchmark the code;

Note that the type-instability that @code_warntype is considered harmless as reported [here](https://discourse.julialang.org/t/how-to-prevent-type-instability-in-for-loops/30508). 

In [2]:
function buildMat1D(N::Int64)
  Nm1::Int64 = N-1; Np1::Int64 = N+1 
  h::Float64 = 1/N; h2::Float64 = h*h; 
  stencil = Vector{Float64}([-1/h2, 2/h2, -1/h2]);
  #..Allocate row, column and value vector 
  I = Int64[]; sizehint!(I, 3*Nm1); 
  J = Int64[]; sizehint!(J, 3*Nm1); 
  vals=Float64[]; sizehint!(vals, 3*Nm1);
  intervalRows = Vector{Int64}(2:N);
  for i in intervalRows 
    append!(I, [i,i,i])
    append!(J, [i-1,i,i+1])
    append!(vals,stencil) 
  end 
  #..Build matrix for interior rows   
  A = sparse(I,J,vals,Np1,Np1)
  #..Build matrix for boundary rows
  A[1,1] = 1; A[end,end]=1; A[2,1] =0; A[end-1,end]=0; 
  return A 
end 

buildMat1D (generic function with 1 method)

In [3]:
A = buildMat1D(4)

5×5 SparseMatrixCSC{Float64, Int64} with 11 stored entries:
 1.0     ⋅      ⋅      ⋅    ⋅ 
 0.0   32.0  -16.0     ⋅    ⋅ 
  ⋅   -16.0   32.0  -16.0   ⋅ 
  ⋅      ⋅   -16.0   32.0  0.0
  ⋅      ⋅      ⋅      ⋅   1.0

In [15]:
B = Matrix(A)

5×5 Matrix{Float64}:
 1.0    0.0    0.0    0.0  0.0
 0.0   32.0  -16.0    0.0  0.0
 0.0  -16.0   32.0  -16.0  0.0
 0.0    0.0  -16.0   32.0  0.0
 0.0    0.0    0.0    0.0  1.0

In [16]:
typeof(B)

Matrix{Float64} (alias for Array{Float64, 2})

In [28]:
@code_warntype buildMat1D(5.);

MethodInstance for buildMat1D(::Float64)
  from buildMat1D(N) in Main at In[21]:2
Arguments
  #self#::Core.Const(buildMat1D)
  N::Float64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  A::SparseMatrixCSC{Float64, Int64}
  intervalRows::Vector{Int64}
  vals::Vector{Float64}
  J::Vector{Int64}
  I::Vector{Int64}
  stencil::Vector{Float64}
  h2::Float64
  h::Float64
  Np1::Int64
  Nm1::Int64
  i::Int64
Body::SparseMatrixCSC{Float64, Int64}
1 ─       Core.NewvarNode(:(A))
│   %2  = (N - 1)::Float64
│   %3  = Base.convert(Main.Int64, %2)::Int64
│         (Nm1 = Core.typeassert(%3, Main.Int64))
│   %5  = (N + 1)::Float64
│   %6  = Base.convert(Main.Int64, %5)::Int64
│         (Np1 = Core.typeassert(%6, Main.Int64))
│   %8  = (1 / N)::Float64
│   %9  = Base.convert(Main.Float64, %8)::Float64
│         (h = Core.typeassert(%9, Main.Float64))
│   %11 = (h * h)::Float64
│   %12 = Base.convert(Main.Float64, %11)::Float64
│         (h2 = Core.typeassert(%12, Main.Float64))
│   %14 = Core.appl

In [25]:
@benchmark buildMat1D(1000)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   87.333 μs …   5.549 ms  ┊ GC (min … max):  0.00% … 98.09%
 Time  (median):      97.166 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   119.481 μs ± 289.167 μs  ┊ GC (mean ± σ):  17.06% ±  6.86%

            ▂▇█▄                                                 
  ▁▂▂▂▂▂▂▂▃▆████▇▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  87.3 μs          Histogram: frequency by time          134 μs <

 Memory estimate: 469.92 KiB, allocs estimate: 2023.

### Section 1.2: Construction of the Right-Hand Side Vector

In [22]:
function buildRhs1D(N::Int64,sourceFct::Function)
  h = 1/N;
  x = Vector(0:h:1)
  #..Build vector for interior rows 
  f = sourceFct(x)
  #..Build matrix for boundary rows
  f[1] = 0; f[end] = 0; 
  return f 
end 

buildRhs1D (generic function with 1 method)

In [20]:
sourceFct(x)= x.*sin.(π*x)

sourceFct (generic function with 1 method)

In [33]:
f = buildRhs1D(5,sourceFct);
typeof(A)

SparseMatrixCSC{Float64, Int64}

In [33]:
@code_warntype buildRhs1D(5,sourceFct)

MethodInstance for buildRhs1D(::Int64, ::typeof(sourceFct))
  from buildRhs1D(N::Int64, sourceFct::Function) in Main at In[30]:1
Arguments
  #self#::Core.Const(buildRhs1D)
  N::Int64
  sourceFct::Core.Const(sourceFct)
Locals
  f::Vector{Float64}
  x::Vector{Float64}
  h::Float64
Body::Vector{Float64}
1 ─      (h = 1 / N)
│   %2 = (0:h:1)::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
│        (x = Main.Vector(%2))
│        (f = (sourceFct)(x))
│        Base.setindex!(f, 0, 1)
│   %6 = f::Vector{Float64}
│   %7 = Base.lastindex(f)::Int64
│        Base.setindex!(%6, 0, %7)
└──      return f



In [34]:
@benchmark buildRhs1D(1000,sourceFct)

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.017 μs …  1.082 ms  ┊ GC (min … max):  0.00% … 99.30%
 Time  (median):     7.542 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.621 μs ± 31.902 μs  ┊ GC (mean ± σ):  12.26% ±  3.29%

  ▁▃▁             ▁▃▄▅▇▇██▇▇▇▆▅▅▄▃▂▁▁                        ▃
  ██████▇▇███▇▆▇▇██████████████████████▇█▆▄▅▄▄▅▅▅▄▂▄▆▆▆▆▇▆▆▆ █
  6.02 μs      Histogram: log(frequency) by time     9.72 μs <

 Memory estimate: 24.00 KiB, allocs estimate: 3.

### Section 1.3: Solve the Linear System - Default Versions 
Here we employ a sparse direct solver. 

In [25]:
function solvePoisson1D(N::Int64,sourceFct::Function)
  A = buildMat1D(N);
  f = buildRhs1D(N,sourceFct)
  u = A\f 
  return u 
end

solvePoisson1D (generic function with 1 method)

In [26]:
@benchmark solvePoisson1D(1000,sourceFct)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  282.958 μs …   9.458 ms  ┊ GC (min … max): 0.00% … 81.18%
 Time  (median):     307.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   356.168 μs ± 490.612 μs  ┊ GC (mean ± σ):  9.81% ±  6.73%

         ▅█▆▄▂                                                   
  ▄▆▄▅▅▅▆██████▇▆▆▅▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▁▁▂ ▃
  283 μs           Histogram: frequency by time          430 μs <

 Memory estimate: 902.95 KiB, allocs estimate: 2087.

In [34]:
# algorithm being used to solve the linear system 
# use edit("/Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:1548")
# to view the source code 
methods(\, (SparseMatrixCSC{Float64, Int64}, Vector{Float64}))

# 1 method for generic function "\":
[1] \(A::SparseArrays.AbstractSparseMatrixCSC, B::AbstractVecOrMat) in SparseArrays at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:1548

### Section 1.4: Solve the Linear System - Alternative Versions 
Here we employ a dense direct solver. 

In [29]:
function solvePoisson1D(N::Int64,sourceFct::Function)
  A = buildMat1D(N);
  B = Matrix(A)
  f = buildRhs1D(N,sourceFct)
  u = B\f 
  return u 
end

solvePoisson1D (generic function with 1 method)

In [30]:
@benchmark solvePoisson1D(1000,sourceFct)

BenchmarkTools.Trial: 184 samples with 1 evaluation.
 Range (min … max):  10.585 ms … 248.194 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.441 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.204 ms ±  29.754 ms  ┊ GC (mean ± σ):  2.50% ± 6.99%

  █▅▅▂▁  ▂ ▃                                                    
  █████▇████▇▇█▆▄▄▆▁▄▄▄▁▁▄▁▁▄▄▆▄▁▄▁▁▁▁▄▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▄
  10.6 ms       Histogram: log(frequency) by time       158 ms <

 Memory estimate: 15.79 MiB, allocs estimate: 2033.

In [35]:
# algorithm being used to solve the linear system 
methods(\, (Matrix{Float64}, Vector{Float64}))

# 1 method for generic function "\":
[1] \(A::AbstractMatrix, B::AbstractVecOrMat) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/generic.jl:1096